In [606]:
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import UTC
import matplotlib.pyplot as plt


In [607]:
df = pd.read_csv("../../db/data/merge/secondary/logs_.csv", index_col=0)
df.index = pd.to_datetime(df.index)
datetime(2022, 7, 1, 0, 0, 0, 0, UTC)

df = df.loc[datetime(2010, 6, 1, 0, 0, 0, 0, UTC):datetime(2022, 7, 1, 0, 0, 0, 0, UTC)]
display(df.head(1), df.tail(1))
df.shape


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2010-06-01 00:00:00+00:00,-0.195535,-0.431203,-0.701336,-0.140981,-0.470019,-0.252915,-0.504554,-0.271715,0.354467,0.168679,...,-0.372762,0.185407,-0.157289,-0.047979,-0.305857,-0.55999,-0.335956,0.293033,0.037194,-0.217444


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,-0.05858,-0.048583,0.019215,0.010852,-0.092832,0.009434,0.079595,0.068199,0.036857,-0.023727,...,0.065983,0.070777,-0.053466,-0.080965,-0.077221,-0.004721,-0.121889,0.041164,0.052337,0.119222


(74593, 28)

In [608]:
SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]



In [609]:
def mean_prediction(df, rate=24):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym}_tend'] = df[sym].rolling(rate).mean()
    data.index = df.index
    return data
        
def random_prediction(df):
    data = pd.DataFrame([])
    
    for sym in SYMBOLS:
        data[f'{sym}']=pd.DataFrame(np.random.uniform(low=-0.8,high=0.8,size=(df.shape[0],1)))
    data.index = df.index
    return data

In [610]:
rand = random_prediction(df)
mu = mean_prediction(df)

In [611]:
def condition(x):
    if x>0:
        return 1
    elif x<0:
        return -1
    else:
        return 0

for sym in mu.columns:
    mu[sym] = mu[sym].map( lambda x: condition(x))
    
# mu.reset_index(inplace=True, drop=True)

display(rand.shape, mu.shape)


(74593, 28)

(74593, 28)

In [612]:
pred_last = rand.iloc[-120:]
pred = rand.iloc[-1:]
pred_last
pred = abs(pred)



In [613]:
def pairs_correlation(df, rate=120):
    df = df.copy()
    data = pd.DataFrame([])
    for pair in range(len(df.columns)):
        for second_pair in range(pair, len(df.columns)):
            data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
    data.index = df.index
    data = data.iloc[-1:]
    return data


def weighted_volatility(volatility):
    weight = pd.DataFrame([])
    volatility = volatility.copy()
    volatility_T = volatility.T
    weight = volatility_T/ volatility_T.mean()
    return  weight

In [614]:
w = weighted_volatility(pred)
wt = w.T

In [615]:
cor = pairs_correlation(pred_last)


/tmp/ipykernel_2334/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_2334/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{pair}_{second_pair}_corr'] = df.iloc[:,pair].rolling(120).corr(df.iloc[:,second_pair])
/tmp/ipykernel_2334/3071229965.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

In [623]:
cor.shape

(1, 406)

In [630]:
cor = cor.iloc[-1:,]

cort = cor.T
cort.columns = ["correlation"]
cort = cort[round(cort.correlation,6) != 1.000000]

display(cort.T, wt)


,0_1_corr,0_2_corr,0_3_corr,0_4_corr,0_5_corr,0_6_corr,0_7_corr,0_8_corr,0_9_corr,0_10_corr,...,23_24_corr,23_25_corr,23_26_corr,23_27_corr,24_25_corr,24_26_corr,24_27_corr,25_26_corr,25_27_corr,26_27_corr
correlation,0.064185,0.033763,-0.225291,0.216548,-0.18812,0.00018,-0.10509,-0.055175,0.014013,-0.040075,...,-0.181465,0.06289,-0.226831,0.008881,-0.101782,0.000395,0.000739,0.008209,-0.106682,0.132689


,AUD_CAD,AUD_CHF,AUD_JPY,AUD_NZD,AUD_USD,CAD_CHF,CAD_JPY,CHF_JPY,EUR_AUD,EUR_CAD,...,GBP_JPY,GBP_NZD,GBP_USD,NZD_CAD,NZD_CHF,NZD_JPY,NZD_USD,USD_CAD,USD_CHF,USD_JPY
2022-07-01 00:00:00+00:00,1.083315,1.820564,1.622259,0.667187,0.520862,0.961126,1.669148,1.442237,0.115317,0.162858,...,1.468902,0.230369,1.044973,0.144809,1.553008,0.973307,1.760646,0.702011,1.599299,1.889289


In [627]:
def risk_volatility(pred_vol, correlations):
    cor_vol = []
    cor_vol_mean = []
    number = len(pred_vol.columns)
    for pair in range(len(pred_vol.columns)):
        # for cor in range(len(correlations.columns)):
        vol = np.array(pred_vol.iloc[:,pair])
        
        data1 = pred_vol.iloc[:,pair+1:].T
        print(data1.shape)
        data2 = correlations.iloc[:,pair*number:number*(pair+1)-1].T
        # print(data2)
        print(data2.shape)
        # print(data2)
        res = data1@data2
        display(res.mean() + vol)
        # cor_vol_mean.append(vol.mean)
    return cor_vol_mean


In [628]:
m = risk_volatility(wt, cort.T)
m


(27, 1)
(27, 1)


ValueError: matrices are not aligned